In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,20):
    print 'scrapping page : ', i
    url = 'https://rubbisheatrubbishgrow.com/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h2', {'class' : 'posttitle'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('time', {'class' : 'entry-date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('section', {'class' : "entry"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  https://rubbisheatrubbishgrow.com/2017/11/01/new-restaurants-nov-2017/
cleaned detail snippet :  alley on number
number fraser street andaz hotel singapore number
facebook 
href :  https://rubbisheatrubbishgrow.com/2017/10/30/best-dish-2017/
cleaned detail snippet :  we are fast approaching the end of the year and we are counting down to the
href :  https://rubbisheatrubbishgrow.com/2017/10/27/88-hong-kong-roast-meat-singapore/
cleaned detail snippet :  for this review of number hong kong roast meat specialist at jalan besar we
href :  https://rubbisheatrubbishgrow.com/2017/10/24/sg-braised-rice-singapore/
cleaned detail snippet :  two weeks ago the people behind thai village restaurant opened a food stall
href :  https://rubbisheatrubbishgrow.com/2017/10/23/orca-business-operating-system/
cleaned detail snippet :  for the past few years the labour crunch in fb industry has impelled the si
href :  https://rubbisheatrubbishgrow.com/2017/10/20/woodlands-sourdo

cleaned detail snippet :  the new lotus kitchen at chinatown point is an outpost of lotus vegetarian 
href :  https://rubbisheatrubbishgrow.com/2017/07/19/imperial-treasures-fine-teochew/
cleaned detail snippet :  asias best number restaurants once put imperial treasure at mbs on its list
href :  https://rubbisheatrubbishgrow.com/2017/07/17/chef-kang-singapore/
cleaned detail snippet :  i was wary before stepping into chef kang at little india because i read on
href :  https://rubbisheatrubbishgrow.com/2017/07/15/alvin-mark-tan/
cleaned detail snippet :  every month we feature a singapore food blog or instagram number to cultiva
href :  https://rubbisheatrubbishgrow.com/2017/07/13/singapore-food-festival-2017/
cleaned detail snippet :  singapore so boring nothing to do this weekend leh eh wrong singapore food 
scrapping page :  8
href :  https://rubbisheatrubbishgrow.com/2017/07/12/tapas-club-singapore/
cleaned detail snippet :  the sister restaurant to the popular yoghurt chain llaoll

cleaned detail snippet :  i dont quite understand the appeal of hainanese curry rice because to me it
href :  https://rubbisheatrubbishgrow.com/2017/04/18/sumo-big-prawn-noodle-amk/
cleaned detail snippet :  one of the biggest hawker trends this year is to increase the price of hawk
scrapping page :  14
href :  https://rubbisheatrubbishgrow.com/2017/04/16/birds-of-a-feather-singapore/
cleaned detail snippet :  you cannot see it from the photo but when you walk into the inner sanctum o
href :  https://rubbisheatrubbishgrow.com/2017/04/14/the-coconut-club-singapore/
cleaned detail snippet :  two muscular gay men were behind us in the queue for the coconut club at an
href :  https://rubbisheatrubbishgrow.com/2017/04/12/new-restaurants-in-singapore-april-2017/
cleaned detail snippet :  this months openings are slightly fewer than marchs number new restaurants 
href :  https://rubbisheatrubbishgrow.com/2017/04/10/po-restaurant-singapore/
cleaned detail snippet :  sadly the po restaurant at 

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 1.42238092422  seconds
133
133


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 6.02361893654  seconds
133
133


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
128,"\n\n\t\t\tBig Fish Small Fish, Punggol: 7 New ...",Punggol has an exciting new dining enclave wit...,"February 2, 2017",big fish small fish punggol new cargo containe...,punggol exciting new dining enclave cargo cont...
129,"\n\n\t\t\tSabar @ Japan Food Town, Wisma Atria...","Initially, we planned to visit another restaur...","February 1, 2017",sabar japan town wisma atrium holy saba set ba...,initially planned visit restaurant packed like...
130,"\n\n\t\t\tSpago by Wolfgang Puck, Marina Bay S...",Spago is Chef Wolfgang Puck’s second restauran...,"January 30, 2017",spago wolfgang puck marina bay sand amazing ca...,spago chef wolfgang puck second restaurant mar...
131,\n\n\t\t\tSankame Tokyo 三亀: One Michelin-Starr...,Sankame at Ginza Tokyo has been established si...,"January 28, 2017",sankame tokyo san gui michelinstarred family r...,sankame ginza tokyo established following famo...
132,"\n\n\t\t\tLittle Bastard, Jalan Besar: The Jon...",Do you remember the 90s when Singapore radio c...,"January 26, 2017",little bastard jalan besar jon snow speakeasy ...,remember 90 radio censored word bitch meredith...


In [8]:
len(df)

133

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

133

In [11]:
x = df.iloc[3,1]
print x

Two weeks ago, the people behind Thai Village Restaurant opened a food stall, SG Braised Rice, in the food court of Ion Orchard, Food Opera. It serves modern ricebowls doused with a fantastic sauce and topped with various premium ingredients. The ricebowls, which range from $6.80 to $18.80, are inspired by a very popular dish, Braised Abalone Rice, from Thai Village. SG Braised Rice shares the same ingredients with Thai Village, so you really do get restaurant quality food at food court prices. The ricebowls consist of: 1. Egg fried rice, wok-fried. 2. Brown Gravy: this is exactly the same brown gravy used at Thai Village. They cook this gravy only upon ordering. It is cooked in individual portions for individual customers, and not in one big pot for convenience sake. First they mix their secret Shiitake mushroom sauce with a real chicken broth slow-brewed from chickens for 6 hours (and not these chicken cube stock). Then they add either prawns or fresh hand-peeled crab meat depending 

In [12]:
# pickle dataframe
df.to_pickle('rubbisheat.pickle')

In [13]:
wd.quit()